#### Github用户数量示例

这是一个有趣的例子。假设我们希望预测Github.com上有多少人注册。Github官方并没有发布最新的统计数据，在最近一次官方公告（2013年1月）中，注册数量为300万。如果我们想要测量今天的数据呢？我们*本来可以* [从以前的公告中推测未来的数字](http://redmonk.com/dberkholz/2013/01/21/github-will-hit-5-million-users-within-a-year/)，但这样使用的数据较少，我们可能会出现数以千计的偏差，基本上只是在拟合复杂的模型。

相反，我们将使用Github实时信息源中的`用户ID`。脚本`github_events.py`将从[Github Public Timeline feed](https://github.com/timeline)中获取最近的300个事件（我们将使用他们的API来访问数据）。从这些事件中，我们提取出与每个事件相关联的`用户ID`。运行下面的脚本并显示一些输出：

In [1]:
%run github_events.py


print "Some User ids from the latest events (push, star, fork etc.) on Github."
print ids[:10]
print
print "Number of unique ids found: ", ids.shape[0]
print "Largest user id: ", ids.max()

KeyError: 'id'

Some User ids from the latest events (push, star, fork etc.) on Github.
[1524995 1978503 1926860 1524995 3707208  374604   37715  770655  502701
 4349707]

Number of unique ids found:  300
Largest user id: 

 2085773151


In [2]:
figsize(12.5, 3)
plt.hist(ids, bins=45, alpha=0.9)
plt.title("Histogram of %d Github User ids" % ids.shape[0]);
plt.xlabel("User id")
plt.ylabel("Frequency");

NameError: name 'figsize' is not defined

一些用户有多个事件，但我们只对唯一的用户id感兴趣，因此我们的id数量少于300个。上面我打印了最大的用户id。这为什么重要？如果Github按顺序分配用户id，这是一个公平的假设，那么我们就**知道**用户数量肯定比那个数字多。请记住，我们只关注远大于300人口中的少数个体，所以我们很可能没有抽取到最近注册的用户。

最好的情况下，我们只能估计注册总数。让我们更加熟悉这个问题。考虑一个我们希望估计用户数量的虚构网站：

1. 假设我们以类似上面的方式抽取了两个人：id分别是3和10。这个网站有可能有数百万用户吗？不太可能。另一方面，这个网站更有可能拥有少于100名用户。

2. 另一方面，如果id是3和34,989，我们可能更愿意猜测可能有数千，甚至数百万的用户注册。由于数据不足，我们对估计并不十分自信。

3. 如果我们抽取了成千上万的用户，最大用户id仍然是34,989，那么总的签约数很可能接近35,000。因此，我们的推断应该更有信心。

我们有以下假设：

**假设：** 每个用户同等可能执行一个事件。显然，从上面的直方图来看，这个假设被违反了。在Github上的参与倾向于早期采用者，这些早期采用者更有可能a）在Github上投入更多，b）更早地看到Github的价值，因此更感兴趣。分布也倾向于新注册用户，他们很可能仅仅为推送一个项目而注册。

创建一个这样的贝叶斯模型很容易。根据上述假设，所有抽样的user_ids都来自一个DiscreteUniform模型，下界为1，上边界未知。我们不确定上边界可能是多少，但我们知道它肯定会大于ids.max()。

使用如此庞大的数字可能会导致数值问题，因此我们将所有内容缩放一百万。因此，我们将使用一个Uniform模型，而不是DiscreteUniform模型：

In [3]:
FACTOR = 1000000.

import pymc as pm

upper_bound = pm.Uniform("n_sign_ups", ids.max() / FACTOR, (ids.max()) / FACTOR + 1)
obs = pm.Uniform("obs", 0, upper_bound, value=ids / FACTOR, observed=True)

# code to be examplained in Chp. 3.
mcmc = pm.MCMC([upper_bound, obs])
mcmc.sample(100000, 45000)

ZeroProbability: Stochastic obs's value is outside its support,
 or it forbids its parents' current values.

In [4]:
from scipy.stats.mstats import mquantiles

samples = mcmc.trace("n_sign_ups")[:]

hist(samples, bins=100,
     label="Uniform prior",
     density=True, alpha=0.8,
     histtype="stepfilled", color="#7A68A6");

quantiles_mean = np.append(mquantiles(samples, [0.05, 0.5, 0.95]), samples.mean())
print "Quantiles: ", quantiles_mean[:3]
print "Mean: ", quantiles_mean[-1]
plt.vlines(quantiles_mean, 0, 33,
           linewidth=2, linestyles=["--", "--", "--", "-"],
           )
plt.title("Posterior distribution of total number of Github users")
plt.xlabel("number of users (in millions)")
plt.legend()
plt.xlim(ids.max() / FACTOR - 0.01, ids.max() / FACTOR + 0.12);

NameError: name 'mcmc' is not defined

以上我们已经绘制出后验分布。请注意，对于用户数量少于`ids.max()`的情况没有分配后验概率。这是好事，因为那将是不可能的情况。

从左到右的三条虚线垂直线分别是5％、50％和95％的分位数线。也就是说，第一条线之前有5％的概率，第二条线之前有50％，第三条线之前有95％。50％的分位数也被称为*中位数*，对于像这样严重倾斜的分布，它是比平均值更好的中心测量。实线是后验分布的均值。

那么我们可以说什么呢？根据上述数据，有95%的可能性用户数量少于440万，并且可能在436万左右。我想知道这个数字有多准确。在撰写本文时，这似乎有点高，考虑到仅仅五个月前，这个数字是300万：

昨晚@github越过了300万用户标记#turntup

我想也许`user_id`参数被宽泛地应用于用户/机器人/更改的名称等，所以我向Github支持团队咨询了此事：

@cmrn_dp 用户ID分配给新用户/组织，无论他们是由人、团体还是机器人控制。

因此，我们可能通过包括组织来过于估计，这些组织可能不应计为用户。待办事项：估算组织数量。谁来接受挑战？